### 문제 1-2: 2단계 체인 만들기 - 영화 추천 시스템 해답

## 환경 설정

In [ ]:
# 필요한 라이브러리 설치
#%pip install -q langchain langchain-openai python-dotenv

In [1]:
# 환경 변수 설정
from dotenv import load_dotenv
import os

# .env 파일에서 API 키 로드
load_dotenv()

True

## 문제 2 해답 구현

### 요구사항
1. 1단계 체인: 장르를 입력받아 영화 1편 추천
2. 2단계 체인: 추천받은 영화의 3줄 줄거리 요약 제공
3. ChatPromptTemplate 사용
4. 두 체인을 LCEL로 연결
5. 각 단계의 결과를 모두 출력하여 과정 확인

In [1]:
# 필요한 라이브러리 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1단계: 장르별 영화 추천 프롬프트
movie_recommendation_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 전문가입니다. 사용자가 요청한 장르에 맞는 영화 1편을 추천하고 간단한 설명을 제공해주세요."),
    ("user", "{genre} 장르의 한국영화 1편을 추천해주세요. 영화 제목과 왜 이 영화를 추천하는지 이유도 함께 알려주세요.")
])

In [2]:
# 2단계: 영화 줄거리 요약 프롬프트
plot_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 줄거리 요약 전문가입니다. 영화의 핵심 내용을 3줄로 간결하게 요약해주세요."),
    ("user", "다음 영화 추천 내용을 바탕으로 해당 영화의 등장인물과 줄거리를 정확히 10줄로 요약해주세요: {movie}")
])

In [3]:
# LLM 모델 초기화
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 1단계 체인: 영화 추천
recommendation_chain = movie_recommendation_prompt | llm | StrOutputParser()
# Step 1: 사용자가 입력한 장르에 따라 영화 추천
movie = recommendation_chain.invoke({"genre": "Drama"})  # 영화 제목 얻기
print(" 추천된 영화:", movie)  # movie 값 출력

# 2단계 체인: 줄거리 요약 (1단계 결과를 입력으로 받음)
plot_chain = (
    {"movie": recommendation_chain}  # 1단계 체인의 결과를 movie로 전달
    | plot_summary_prompt
    | llm
    | StrOutputParser()
)

 추천된 영화: '올드보이' (2003)

올드보이는 드라마 장르의 한국 영화로, 감독 박찬욱이 연출한 작품입니다. 이 영화는 2003년에 개봉되어 많은 사랑을 받았습니다. 

올드보이는 주인공 오대수(최민식)가 15년 동안 감금된 후 탈출하여 복수를 추구하는 내용을 다루고 있습니다. 이 영화는 감동적인 이야기, 뛰어난 연기력, 그리고 깊은 메시지로 많은 팬들을 사로잡았습니다.

올드보이는 한국 영화의 대표작 중 하나로 꼽히며, 세계적으로도 인정받은 작품입니다. 이 영화는 드라마 장르의 팬들에게 꼭 추천하고 싶은 영화입니다.


In [4]:
# 테스트 실행
try:
    genre = "Drama"
    
    print("영화 추천 시스템")
    print("==================")
    print(f"\n입력 장르: {genre}")
    
    # 1단계 실행 및 결과 출력
    movie_recommendation = recommendation_chain.invoke({"genre": genre})
    print("\n1단계 - 영화 추천 결과:")
    print(movie_recommendation)
    
    print("\n---\n")
    
    # 2단계 실행 및 결과 출력
    plot_summary = plot_chain.invoke({"genre": genre})
    print("2단계 - 영화 줄거리 요약:")
    print(plot_summary)
    
except Exception as e:
    print(f"오류 발생: {e}")

영화 추천 시스템

입력 장르: Drama

1단계 - 영화 추천 결과:
**영화:** "올드보이" (2003)

**추천 이유:** "올드보이"는 한국 영화계의 대표작 중 하나로, 김지운 감독이 연출한 드라마/스릴러 영화입니다. 이 영화는 주인공 오대수(최민식)가 15년 동안 감금된 후 풀려나면서 시작되는 이야기를 다루고 있습니다. 오대수는 자신의 과거와 감금된 이유를 찾기 위해 노력하며, 그 과정에서 많은 반전과 충격을 주는 스토리로 구성되어 있습니다.

이 영화는 뛰어난 연기력과 긴장감 있는 전개, 그리고 사회 비판적인 메시지를 전달하는 점에서 높이 평가받고 있습니다. 또한, "올드보이"는 한국 영화의 국제적인 위상을 높인 작품 중 하나로, 칸 영화제에서 심사위원 대상을 수상하기도 했습니다.

따라서, 드라마 장르의 한국 영화를 찾고 있다면 "올드보이"는 강력히 추천할 만한 영화입니다.

---

2단계 - 영화 줄거리 요약:
올드보이는 2003년에 개봉한 한국 드라마 영화입니다. 

주인공 오대수는 15년간 아무런 이유 없이 감금된 채로 살아갑니다. 

오대수는 자신의 과거와 감금의 이유를 모른 채로 고통스러운 시간을 보냅니다. 

그러던 중, 오대수는 감금된 방을 탈출하고, 자신의 과거와 감금의 진실을 찾기 시작합니다. 

그는 자신의 감금과 관련이 있는 미스터리한 여성을 만나면서 복수를 준비합니다. 

오대수는 자신의 감금과 복수의 진실을 밝히기 위해 노력합니다. 

그는 과거의 상처와 고통을 극복하고, 진실을 찾기 위해 노력합니다. 

오대수의 이야기는 복수와 용서의 주제로 깊은 감동을 전달합니다. 

올드보이는 배우들의 연기력과 뛰어난 각본, 감독의 연출력이 돋보이는 작품입니다. 

올드보이는 한국 영화계에서 큰 인기를 얻었고, 해외에서도 많은 관심을 받았습니다.


## 추가 테스트

In [5]:
# 다른 장르로 추가 테스트
genre2 = "로맨스"

movie_recommendation2 = recommendation_chain.invoke({"genre": genre2})
plot_summary2 = plot_chain.invoke({"genre": genre2})

print(f"다른 장르로 테스트 - {genre2}:")
print("===============================")
print("\n영화 추천:")
print(movie_recommendation2)
print("\n줄거리 요약:")
print(plot_summary2)

다른 장르로 테스트 - 로맨스:

영화 추천:
**'건축학개론'**

'건축학개론'은 2012년에 개봉한 한국 영화로, 로맨스 장르에 속합니다. 이 영화는 대학생 시절 첫사랑을 뒤늦게 다시 만나게 되면서 일어나는 감동적인 이야기를 다루고 있습니다. 영화는 두 주인공이 겪는 우연한 재회와 과거의 추억, 그리고 현재의 감정을 섬세하게 표현하여 많은 관객들에게 사랑을 받았습니다. 특히, 감독의 세심한 연출과 배우들의 연기가 돋보이며, 한국 영화의 대표작 중 하나로 꼽힙니다. 로맨틱한 이야기와 감동적인 연기를 원한다면 '건축학개론'을 추천합니다.

줄거리 요약:
Here is a 10-line summary of the movie "어바웃 타임":

*   주인공 '태희'는 시간여행을 통해 과거로 돌아가 자신의 현재를 바꾸려고 합니다.
*   그는 과거로 돌아가면서도 자신의 현재를 바꿀 수 없다는 것을 깨닫습니다.
*   태희는 결국 자신의 사랑을 찾기 위해 시간여행을 계속합니다.
*   영화는 로맨틱한 분위기와 감동적인 결말로 많은 사랑을 받았습니다.
*   태희는 과거에서 자신의 첫사랑 '수진'을 만나게 됩니다.
*   태희와 수진은 서로에 대한 사랑을 느끼지만, 태희는 미래에서 온 사람이라는 이유로 수진과 멀어지게 됩니다.
*   태희는 수진과의 사랑을 찾기 위해 계속해서 시간여행을 합니다.
*   영화는 태희와 수진의 사랑 이야기로 인해 많은 관객에게 감동과 설렘을 선사합니다.
*   태희는 시간여행을 하면서도 자신의 현재와 과거를 되돌아보게 됩니다.
*   결국, 태희는 수진과의 사랑을 찾고, 영화는 감동적인 결말로 마무리됩니다.


## 체인 연결 패턴 설명

In [ ]:
# 체인 연결 방식 설명
print("체인 연결 방식 설명:")
print("====================")
print()
print("1단계 체인:")
print("movie_recommendation_prompt | llm | StrOutputParser()")
print()
print("2단계 체인:")
print('{"movie": recommendation_chain} | plot_summary_prompt | llm | StrOutputParser()')
print()
print("핵심 포인트:")
print("- 1단계 체인의 출력이 2단계 체인의 입력으로 자동 전달")
print("- movie 키를 통해 데이터 흐름 제어")
print("- LCEL을 통해 직관적인 파이프라인 구성")

체인 연결 방식 설명:

1단계 체인:
movie_recommendation_prompt | llm | StrOutputParser()

2단계 체인:
{"movie_info": recommendation_chain} | plot_summary_prompt | llm | StrOutputParser()

핵심 포인트:
- 1단계 체인의 출력이 2단계 체인의 입력으로 자동 전달
- movie_info 키를 통해 데이터 흐름 제어
- LCEL을 통해 직관적인 파이프라인 구성


## 학습 정리

### 문제 2에서 배운 핵심 개념
1. **ChatPromptTemplate**: 시스템과 사용자 메시지 구조화
2. **다단계 체인**: 여러 단계로 나누어 복잡한 작업 처리
3. **체인 간 데이터 전달**: {"key": previous_chain} 패턴
4. **단계별 결과 확인**: 각 체인의 중간 결과 출력

### 구현 완료 사항
- ✅ 2단계 체인 구현
- ✅ ChatPromptTemplate 사용
- ✅ 체인 간 데이터 전달
- ✅ 영화 추천 기능
- ✅ 줄거리 요약 기능
- ✅ 각 단계별 결과 출력
- ✅ 다양한 장르 테스트

### 체인 연결의 핵심 패턴
```python
# 기본 패턴
chain1 = prompt1 | llm | parser
chain2 = {"input_key": chain1} | prompt2 | llm | parser
```

이 패턴을 통해 복잡한 작업을 단계별로 분해하여 처리할 수 있습니다.